In [1]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

C:\Users\Big data\PycharmProjects\DB103\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Big data\PycharmProjects\DB103\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Big data\PycharmProjects\DB103\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Big data\PycharmProjects\DB103\venv\l

In [2]:
#建立分類資料夾
#C:/Users/Big data/Desktop/pic_data/sample/valid/document
#C:/Users/Big data/Desktop/pic_data/sample/valid/graduation
#C:/Users/Big data/Desktop/pic_data/sample//valid/live
#C:/Users/Big data/Desktop/pic_data/sample/train/document
#C:/Users/Big data/Desktop/pic_data/sample/train/graduation
#C:/Users/Big data/Desktop/pic_data/sample/train/live

In [4]:
# 資料路徑
DATASET_PATH  = 'C:/Users/Big data/Desktop/pic_data/sample'

# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 3

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 8

# 凍結網路層數
FREEZE_LAYERS = 2

# Epoch 數
NUM_EPOCHS = 10

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'

# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

Found 126 images belonging to 3 classes.
Found 1069 images belonging to 3 classes.
Class #0 = document
Class #1 = graductation
Class #2 = live


C:\Users\Big data\PycharmProjects\DB103\venv\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
# 輸出整個網路結構
print(net_final.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
______________________________________________________________________________________________

In [6]:
# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

Epoch 1/10
14/15 [===========================>..] - ETA: 5s - loss: 2.5045 - acc: 0.3182 

C:\Users\Big data\PycharmProjects\DB103\venv\lib\site-packages\PIL\Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


15/15 [==============================] - 139s 9s/step - loss: 2.4549 - acc: 0.3136 - val_loss: 2.0380 - val_acc: 0.3402
Epoch 2/10
15/15 [==============================] - 123s 8s/step - loss: 2.2301 - acc: 0.4068 - val_loss: 1.8150 - val_acc: 0.4408
Epoch 3/10
15/15 [==============================] - 123s 8s/step - loss: 1.6878 - acc: 0.4915 - val_loss: 1.4254 - val_acc: 0.5423
Epoch 4/10
15/15 [==============================] - 122s 8s/step - loss: 1.4929 - acc: 0.5932 - val_loss: 1.3491 - val_acc: 0.5930
Epoch 5/10
15/15 [==============================] - 124s 8s/step - loss: 1.4260 - acc: 0.6083 - val_loss: 1.1041 - val_acc: 0.6419
Epoch 6/10
15/15 [==============================] - 122s 8s/step - loss: 0.9846 - acc: 0.6810 - val_loss: 1.0165 - val_acc: 0.6814
Epoch 7/10
15/15 [==============================] - 123s 8s/step - loss: 1.3486 - acc: 0.6525 - val_loss: 0.9734 - val_acc: 0.6983
Epoch 8/10
15/15 [==============================] - 123s 8s/step - loss: 0.9071 - acc: 0.6917 

In [8]:
# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

In [9]:
'''
import glob
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing import image
import numpy as np
#輸入圖片資料夾路徑
files = glob.glob("C:/Users/Big data/Desktop/pic_data/test/*.jpg")
# 載入訓練好的模型
net = load_model('model-resnet50-final.h5')
cls_list = ['document', 'graduation', 'live']

# 辨識每一張圖
for f in files:
    img = image.load_img(f, target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    print(f)
    for i in top_inds:
        #print('    {:.3f}  {}'.format(pred[i], cls_list[i]))
        print(pred[i], cls_list[i])
'''

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


FileNotFoundError: [Errno 2] No such file or directory: '-f'